In [ ]:
import torch, time, gc, os
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialise training variables and datasets, split into train, validation and test sets.

In [ ]:
# Training variables
BATCH_SIZE = 32
EPOCHS = 150
LEARNING_RATE = 0.00005

In [ ]:
# Transformations for the train, validation, and test datasets
train_transform = transforms.Compose([
    transforms.Resize(256, antialias=True),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((256, 256), antialias=True),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the train, validation, and test datasets
train_dataset = datasets.Flowers102(root="./data", split="train", transform=train_transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_dataset = datasets.Flowers102(root="./data", split="val", transform=val_test_transform, download=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_dataset = datasets.Flowers102(root="./data", split="test", transform=val_test_transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Define CNN model.

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Conv -> batch normalization -> relu -> pool
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # output -> 112 x 112 x 16
        ) 
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # output -> 56 x 56 x 32
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # output -> 28 x 28 x 64
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # output -> 14 x 14 x 128
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # output -> 7 x 7 x 256
        )
        self.conv6 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # output -> 3 x 3 x 512
        )
        self.conv7 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # output -> 1 x 1 x 1024
        )

        # Linear -> relu
        self.fc1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU()
        )
        self.fc3 = nn.Sequential(
            nn.Linear(256, 102)
        )
      
     def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = x.view(-1, 1024)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

# Function to save the model with the best validation accuracy.

In [ ]:
def save_network(network, epoch_label):
    save_filename = 'net_%s.pth' % epoch_label
    save_path = os.path.join('./savedModels', save_filename)
    torch.save(network.cpu().state_dict(), save_path)
    if torch.cuda.is_available():
       network.cuda()

# Initialise CNN instance and train.

In [ ]:
# Clear CUDA cache
torch.cuda.empty_cache()

In [ ]:
# Create an instance of the CNN and move it to the device
cnn = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.1, verbose=True)

In [ ]:
start_time = time.time()
best_epoch = 0
best_acc = 0

# Train the CNN
for epoch in range(EPOCHS):
    print(f'--------- Epoch {epoch + 1} ----------')
    cnn.train()
    train_loss = 0
    train_correct = 0
    train_total = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = cnn(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_loss /= len(train_loader)
    train_accuracy = 100 * train_correct / train_total
    print(f'  Training Accuracy: {train_accuracy:.2f}%')

 # Evaluate model after each training epoch
    cnn.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = cnn(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    accuracy = 100 * correct / total
    print(f'  Validation Accuracy: {accuracy:.2f}%')

    # Update the learning rate scheduler
    scheduler.step(val_loss)

     if accuracy > best_acc:
      best_acc = accuracy
      torch.save(cnn.state_dict(), 'best-model-parameters.pt')
      best_epoch = epoch + 1
      print("  **Better model found. Updated best model.")

# Test model.

In [ ]:
# Test the best model on the test set
best_model = torch.load("best-model-parameters.pt")
cnn.load_state_dict(best_model)
print("---------- Best Model Testing ----------")
print(f'  Epoch: {best_epoch}')
print(f'  Validation accuracy: {best_acc:.2f}%')

cnn.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'  Test Accuracy: {test_accuracy:.2f}%')